In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout, row):
        super.__init__(self)
        
        # if row is True, possibility maps from rows are given as input
        if row is True:
            kernel_size = (1,3)
            padding = (0, 1)

        else:
            kernel_size = (3, 1)
            padding = (1, 0)
            
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=padding, dilation=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout2d(dropout)

        
    def forward(self, x):

        identity = x

        x = self.conv(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv(x)
        x = self.bn1(x)

        x += identity

        x = self.relu(x)
        x = self.dropout(x)

        return x


In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super.__init__(self)

        self.rclayer1 = ConvBlock(in_channels = 1, out_channels=32, dropout = 0.1, row = True)
        self.rclayer2 = ConvBlock(in_channels = 32, out_channels = 64, dropout = 0.1, row = True)
        self.rclayer3 = ConvBlock(in_channels = 64, out_channels = 128, dropout = 0.1, row = True)
        self.rclayer4 = ConvBlock(in_channels = 128, out_channels = 256, dropout = 0.1, row = True)

        self.cclayer1 = ConvBlock(inchannels = 1, out_channels= 32, dropout = 0.1, row = False)
        self.cclayer2 = ConvBlock(inchannels = 32, out_channels= 64, dropout = 0.1, row = False)
        self.cclayer3 = ConvBlock(inchannels = 64, out_channels= 128, dropout = 0.1, row = False)
        self.cclayer4 = ConvBlock(inchannels = 128, out_channels= 256, dropout = 0.1, row = False)

    def forward(self, x):
        assert x.ndim == 4 # Shape (Batch, Channel = 2, Height, Width)

        row_x = x[:,0,:,:]
        column_x = x[:,1,:,:]

        print(row_x)
        print(column_x)

        row_x = self.rclayer1(x)
        row_x = self.rclayer2(x)
        row_x = self.rclayer3(x)
        row_x = self.rclayer4(x)

        column_x = self.cclayer1(x)
        column_x = self.cclayer2(x)
        column_x = self.cclayer3(x)
        column_x = self.cclayer4(x)

        return row_x, column_x



NameError: name 'nn' is not defined

In [ ]:
class PSPModule(nn.Module):
    def __init__(self, features, out_features=1024, sizes=(1, 2, 3, 6)):
        super().__init__()
        self.stages = []
        self.stages = nn.ModuleList([self._make_stage(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * (len(sizes) + 1), out_features, kernel_size=1)
        self.relu = nn.ReLU()

    def _make_stage(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        priors = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear') for stage in self.stages] + [feats]
        bottle = self.bottleneck(torch.cat(priors, 1))
        return self.relu(bottle)

In [ ]:
class PSPUpsample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU()
        )

    def forward(self, x):
        h, w = 2 * x.size(2), 2 * x.size(3)
        p = F.upsample(input=x, size=(h, w), mode='bilinear')
        return self.conv(p)


In [ ]:
class PSPNet(nn.Module):
    def __init__(self, n_classes=18, sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet34',
                 pretrained=True):
        super().__init__()
        self.feats = getattr(extractors, backend)(pretrained)
        self.psp = PSPModule(psp_size, 1024, sizes)
        self.drop_1 = nn.Dropout2d(p=0.3)

        self.up_1 = PSPUpsample(1024, 256)
        self.up_2 = PSPUpsample(256, 64)
        self.up_3 = PSPUpsample(64, 64)

        self.drop_2 = nn.Dropout2d(p=0.15)
        self.final = nn.Sequential(
            nn.Conv2d(64, n_classes, kernel_size=1),
            nn.LogSoftmax()
        )

        self.classifier = nn.Sequential(
            nn.Linear(deep_features_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes)
        )

    def forward(self, x):
        f, class_f = self.feats(x) 
        p = self.psp(f)
        p = self.drop_1(p)

        p = self.up_1(p)
        p = self.drop_2(p)

        p = self.up_2(p)
        p = self.drop_2(p)

        p = self.up_3(p)
        p = self.drop_2(p)

        auxiliary = F.adaptive_max_pool2d(input=class_f, output_size=(1, 1)).view(-1, class_f.size(1))

        return self.final(p), self.classifier(auxiliary)